In [1]:
import time
import pymssql
import pandas as pd
import urllib
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from pangres import upsert

In [2]:
# driver = 'SQL Server Native Client 10.0'
# server = 'xxxxx'　 # データベースの詳細設定のサーバー名
# database = 'xxxxx' # データベース名
# username = 'xxxxx' # データベースの詳細設定のサーバー管理者ログイン
# password = 'xxxxx' # データベースの詳細設定の更新

In [4]:
engine = create_engine(f"mssql+pymssql://{username}:{password}@{server}/{database}")

In [5]:
with engine.connect() as conn:
    rs = conn.execute('SELECT @@VERSION as version')
    for row in rs:
        print(row['version'])

Microsoft SQL Azure (RTM) - 12.0.2000.8 
	Sep 18 2021 19:01:34 
	Copyright (C) 2019 Microsoft Corporation



## テーブルの定義

In [6]:
Base = declarative_base()

In [7]:
class Person(Base):
    __tablename__ = 'persons'
    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String(14))
    job = Column(String(14))
    address = Column(String(50))

## 登録する DataFrame型のデータを作成

In [22]:
df = pd.DataFrame(
    [
        [1, "John1", "Engineer", "America"],
        [2, "John2", "Engineer", "America"],
        [3, "John3", "Engineer", "America"],
        [4, "John4", "Engineer", "America"],
        [5, "John5", "Engineer", "America"],
        [6, "John6", "Engineer", "America"],
        [7, "John7", "Engineer", "America"],
        [8, "John8", "Engineer", "America"],
        [9, "John9", "Engineer", "America"],
        [10, "John10", "Engineer", "America"],
        [11, "John11", "Engineer", "America"],
        [12, "John12", "Engineer", "America"],
        [13, "John13", "Engineer", "America"],
        [14, "John14", "Engineer", "America"],
        [15, "John15", "Engineer", "America"],
        [16, "John16", "Engineer", "America"],
        [17, "John17", "Engineer", "America"],
        [18, "John18", "Engineer", "America"],
        [19, "John19", "Engineer", "America"],
        [20, "John20", "Engineer", "America"],
        [21, "John21", "Engineer", "America"],
        [22, "John22", "Engineer", "America"],
        [23, "John23", "Engineer", "America"],
        [24, "John24", "Engineer", "America"],
        [25, "John25", "Engineer", "America"],
        [26, "John26", "Engineer", "America"],
        [27, "John27", "Engineer", "America"],
        [28, "John28", "Engineer", "America"],
        [29, "John29", "Engineer", "America"],
        [30, "John30", "Engineer", "America"],
    ],
    columns = ["id", "name", "job", "address"]
)

df.head()

,id,name,job,address
0,1,John1,Engineer,America
1,2,John2,Engineer,America
2,3,John3,Engineer,America
3,4,John4,Engineer,America
4,5,John5,Engineer,America


## for文で1つずつ登録

In [9]:
# テーブルの作成
Base.metadata.create_all(engine)

In [10]:
start = time.time()

Session = sessionmaker(bind=engine)
session = Session()

for i in range(len(df)):
    
    p = Person(
        name = df.iloc[i:i+1]["name"].values[0],
        job = df.iloc[i:i+1]["job"].values[0],
        address = df.iloc[i:i+1]["address"].values[0]
    )
    
    session.add(p)
    session.commit()

session.flush()

for_add_time = time.time() - start
print(f"elapsed_time:{round((for_add_time), 1)}[sec]")

elapsed_time:11.3[sec]


In [11]:
# テーブルの削除
Base.metadata.drop_all(engine)

## to_sql で登録

In [12]:
# テーブルの作成
Base.metadata.create_all(engine)

In [13]:
start = time.time()

df[["name", "job", "address"]].to_sql('persons', engine, if_exists='append', index=False)

to_sql_time = time.time() - start
print(f"elapsed_time:{round((to_sql_time), 1)}[sec]")

elapsed_time:2.9[sec]


In [14]:
print(f"for文で1行ずつ登録:{round(for_add_time, 1)}秒")
print(f"to_sql で登録:{round(to_sql_time, 1)}秒")

for文で1行ずつ登録:11.3秒
to_sql で登録:2.9秒


## for文で 1つずつ Update

In [15]:
df_update = df.copy()

df_update["name"] = "update"
df_update["job"] = "update"
df_update["address"] = "update"

df_update.head()

,id,name,job,address
0,1,update,update,update
1,2,update,update,update
2,3,update,update,update
3,4,update,update,update
4,5,update,update,update


In [16]:
start = time.time()

Session = sessionmaker(bind=engine)
session = Session()

for i in range(len(df_update)):
    
    id_num = int(df_update.iloc[i:i+1].id.values[0])
    mydata = session.query(Person).filter(Person.id == id_num).first()
    
    mydata.name = df_update.iloc[i:i+1]["name"].values[0]
    mydata.job = df_update.iloc[i:i+1]["job"].values[0]
    mydata.address = df_update.iloc[i:i+1]["address"].values[0]
    
    session.add(mydata)
    session.commit()
    
session.flush()

for_update_time = time.time() - start
print(f"elapsed_time:{round((for_update_time), 1)}[sec]")

elapsed_time:13.2[sec]
